In [1]:
from src.data_processing.dataset import LmdbDataset, Database
from src.net.modules.encoder import ViTEncoder, ViTSTR
from src.data_processing.vocabulary import Vocabulary, TokenLabelConverter

In [2]:
vocab = Vocabulary()
vocab.numericalize('Hello')

tensor([ 1, 20, 17, 24, 24, 27,  2])

In [8]:
tlc = TokenLabelConverter()
tlc.encode('HeloloPRIVET')

tensor([[ 0, 19,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 16,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 23,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 26,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 23,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 26,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 0, 27,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0, 

In [4]:
tlc.dict

{'[GO]': 0,
 '[s]': 1,
 '0': 2,
 '1': 3,
 '2': 4,
 '3': 5,
 '4': 6,
 '5': 7,
 '6': 8,
 '7': 9,
 '8': 10,
 '9': 11,
 'a': 12,
 'b': 13,
 'c': 14,
 'd': 15,
 'e': 16,
 'f': 17,
 'g': 18,
 'h': 19,
 'i': 20,
 'j': 21,
 'k': 22,
 'l': 23,
 'm': 24,
 'n': 25,
 'o': 26,
 'p': 27,
 'q': 28,
 'r': 29,
 's': 30,
 't': 31,
 'u': 32,
 'v': 33,
 'w': 34,
 'x': 35,
 'y': 36,
 'z': 37,
 '!': 38,
 '"': 39,
 '#': 40,
 '$': 41,
 '%': 42,
 '&': 43,
 "'": 44,
 '(': 45,
 ')': 46,
 '*': 47,
 '+': 48,
 ',': 49,
 '-': 50,
 '.': 51,
 '/': 52,
 ':': 53,
 ';': 54,
 '<': 55,
 '=': 56,
 '>': 57,
 '?': 58,
 '@': 59,
 '[': 60,
 '\\': 61,
 ']': 62,
 '^': 63,
 '_': 64,
 '`': 65,
 '{': 66,
 '|': 67,
 '}': 68,
 '~': 69}

In [5]:
tlc.decode(tlc.encode('hello'), [0, 0, 0, 0])

['[GO]h[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 '[GO]e[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 '[GO]l[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]',
 '[GO]l[s][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO][GO]']

In [6]:
vocab = Vocabulary()
db = Database('/mnt/s/CV/data_lmdb_release/training/ST/')
ds = LmdbDataset(db, vocab)
encoder = ViTSTR(len(vocab))

In [7]:
encoder(ds[0][0].unsqueeze(0)).shape

AssertionError: Input height (224) doesn't match model (72).